In [1]:
import numpy as np
import json
import os
import cv2
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from pathlib import Path
import pandas as pd 
import numpy as np
import glob
import fiona
from albumentations import Rotate
import torch
import warnings
warnings.filterwarnings("ignore")
import json
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
import segmentation_models_pytorch as smp
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
from torchvision.utils import draw_segmentation_masks


In [5]:
main_train_path = 'content_train/train/'

In [6]:
# Функция создает директорию, если ее не существует по указанному пути
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [7]:
# Функция удаляет пробелы в именах файлов, если такие есть
def del_space(train_path):
    for file in sorted(os.listdir(train_path)):
        if ' ' in file:
            os.rename(str(train_path)+file , str(train_path)+file.replace(" ", ""))
            print(file + ' переименован в ' + str(train_path)+file.replace(" ", ""))
            
# функция находит '.png' файлы, для которых нет соответствующих '.geojson' файлов,
# и наоборот: находит '.geojson' файлы без соответствующих '.png' файлов
def find_waste_files(train_path):
    del_list = []
    for file in sorted(os.listdir(train_path)):
        if file[-3:] == 'png':
            if (os.path.exists(str(train_path)+file[:-3]+'geojson'))!=True:
                del_list.append(str(train_path)+file[:-3]+'png')
        if file[-7:] == 'geojson':
            if (os.path.exists(str(train_path)+file[:-7]+'png'))!=True:
                del_list.append(str(train_path)+file[:-7]+'geojson')
    return del_list

In [8]:
# Переименовываем файлы с пробелами
del_space(main_train_path)

In [9]:
# Получаем список фотографий, для которых нет масок, а также - масок без соответствующих фото 
del_list = find_waste_files(main_train_path)

In [83]:
print(del_list)

['content_train/train/113.geojson', 'content_train/train/176.png', 'content_train/train/257.png', 'content_train/train/393.geojson', 'content_train/train/527.geojson', 'content_train/train/647.png', 'content_train/train/688.geojson', 'content_train/train/72.geojson']


In [84]:
# удаляем лишние файлы
for file in del_list:
    os.remove(file)
    print("Файл " + file + " удален")

Файл content_train/train/113.geojson удален
Файл content_train/train/176.png удален
Файл content_train/train/257.png удален
Файл content_train/train/393.geojson удален
Файл content_train/train/527.geojson удален
Файл content_train/train/647.png удален
Файл content_train/train/688.geojson удален
Файл content_train/train/72.geojson удален


In [85]:
# Функция разрезает изображение с шириной width и высотой height на 8 частей:
# по ширине делим на h_parts=4 частей, по высоте - на v_parts=2 части
def crop_images(main_path, images_path, parts_path, im_list=[], height=1232, 
                width=1624, h_parts=4, v_parts=2):
    for file in sorted(os.listdir(main_path)):
        if file[-3:] == 'png':
            path = images_path + file
            path2 = parts_path + file[:-4]
            image = cv2.imread(str(path), cv2.IMREAD_COLOR)
            image = np.array(image)
            for i in range(0, v_parts):
                for j in range(0, h_parts):
                    v_from = int(i*(height/v_parts))
                    v_to = int((i+1)*(height/v_parts))
                    h_from = int(j*(width/h_parts))
                    h_to = int((j+1)*(width/h_parts))
                    im1 = image[v_from:v_to, h_from:h_to]
                    cv2.imwrite(path2+'_' + str((i*4)+(j+1)) + '.png', im1)

In [11]:
# Определим пути к следующим директориям:
# для разрезанных изображений трейна
train_parts_images = 'content_train/parts_images/'
# для изображений с наложенными масками
masks_with_images_path = 'content_train/masks_with_images/'
# для масок трейна
train_masks_path = 'content_train/masks/'
# для разрезанных масок трейна
train_parts_masks = 'content_train/parts_masks/'
# для изображений после аугментации
aug_images = 'content_train/aug_images/'
# для масок после аугментации
aug_masks = 'content_train/aug_masks/'

In [96]:
# Создадим необходимые директории
create_dir('content_train/parts_images')
create_dir('content_train/masks')
create_dir('content_train/parts_masks')
create_dir('content_train/masks_with_images')
create_dir('content_train/aug_masks')
create_dir('content_train/aug_images')

## Маски

In [98]:
# Вспомогательные функции:

In [2]:
def read_layout(path: str, image_size: tuple) -> np.ndarray:
    """
    Метод для чтения geojson разметки и перевода в numpy маску
    """
    with open(path, 'r', encoding='cp1251') as f:  # some files contain cyrillic letters, thus cp1251
        json_contents = json.load(f)

    num_channels = 2
    mask_channels = [np.zeros(image_size, dtype=np.float32) for _ in range(num_channels)]
    mask = np.zeros(image_size, dtype=np.float32)

    if type(json_contents) == dict and json_contents['type'] == 'FeatureCollection':
        features = json_contents['features']
    elif type(json_contents) == list:
        features = json_contents
    else:
        features = [json_contents]

    for shape in features:
        channel_id = 1
        mask = parse_mask(shape['geometry'], image_size)
        mask_channels[channel_id] = np.maximum(mask_channels[channel_id], mask)

    mask_channels[0] = 1 - np.max(mask_channels[1:], axis=0)

    return np.stack(mask_channels, axis=-1)

In [3]:
def parse_polygon(coordinates, image_size): 
    mask = np.zeros(image_size, dtype=np.float32) 
    
    if len(coordinates) == 1: 
        points = [np.int32(coordinates)] 
        cv2.fillPoly(mask, points, 1) 
    else: 
        points = [np.int32([coordinates[0]])] 
        cv2.fillPoly(mask, points, 1) 
    
        for polygon in coordinates[1:]: 
            points = [np.int32([polygon])] 
            cv2.fillPoly(mask, points, 0) 
    
    return mask


def parse_mask(shape, image_size):
    """
    Метод для парсинга фигур из geojson файла
    """
    mask = np.zeros(image_size, dtype=np.float32)
    coordinates = shape['coordinates']
    if shape['type'] == 'MultiPolygon':
        for polygon in coordinates:
            mask += parse_polygon(polygon, image_size)
    else:
        mask += parse_polygon(coordinates, image_size)

    return mask

In [12]:
# Преобразуем маски в png и сохраним в директории "content_train/masks"
for file in os.listdir(main_train_path):
    if file[-7:] == 'geojson':
        # Получаем соответствующий файл разметки
        json_path = main_train_path +str(file)
        mask = read_layout(json_path, (1232, 1624))
        image = cv2.imread(main_train_path + str(file[:-7])+'png', cv2.IMREAD_COLOR)
        if image.shape[0] == 1232 and image.shape[1]== 1624:
            image = np.array(image / 255, dtype=np.float32)
            transform = transforms.Compose([
                transforms.ToTensor()
            ])
            image = transform(image)
            tensor_mask = transform(mask)
            image_with_mask = draw_segmentation_masks((image.cpu() * 255).type(torch.uint8),
                                                      tensor_mask.type(torch.bool), alpha=0.2)
            image_with_mask = np.moveaxis(image_with_mask.cpu().numpy(), 0, -1)
            cv2.imwrite(masks_with_images_path + str(file[:-7])+'png', image_with_mask)
            mask = 255*mask[:, :,-1]
            cv2.imwrite(train_masks_path + str(file[:-7])+'png', mask)
        else:
            print(file, image.shape)

206.geojson (947, 1248, 3)


In [111]:
# На основе анализа фотографий с наложенными на них масками, расположенными в директории
# 'content_train/masks_with_images/', получаем номера изображений маски для которых
# могут быть некорректными
del_list = [206,9,14,29,36,38,42,44,46,51,52,63,70,85,91,96,111,114,126,138,155,178,
           179,181,188,191,197,198,199,209,211,223,253,262,264,269,272,275,277,279,
           280,284,298,347,349,370,374,383,404,410,411,420,440,488,504,530,531,537,
           540,542,543,546,559,560,562,569573,579,587,589,591,593,606,607,612,621,
           622,638,645,648,653,657,671,673,677,681,685,689,712,715,716,726,728,741,
           744,745,771,778]

In [112]:
# удалим лишние файлы из директории, содержащей маски 
for file in sorted(os.listdir(train_masks_path)):
    if int(file[:-4]) in del_list:
        os.remove(train_masks_path + file)

In [113]:
len(os.listdir(train_masks_path))

556

In [120]:
# разрежем изображения и маски трейна
crop_images(train_masks_path, train_masks_path, train_parts_masks)
crop_images(train_masks_path, main_train_path, train_parts_images)

In [121]:
# создадим директории для разрезанных изображении и масок теста
create_dir('content/test/parts_images')
create_dir('content/test/parts_masks')

In [116]:
test_path = 'content/test/eyes/'
test_path_parts = 'content/test/parts_images/'
# разрежем изображения теста
crop_images(test_path, test_path, test_path_parts)

In [122]:
# сравним число изображений и масок трейновых данных
len(os.listdir(train_parts_masks)),len(os.listdir(train_parts_images))

(4448, 4448)

## Аугментация

In [123]:
# формируем списки с изображениями и масками
def make_train_test_data(images_path, masks_path):
    data_x = []
    data_y = []
    for file in os.listdir(masks_path):
        #print(file[2:-6])
        if file in os.listdir(images_path):
            data_x.append(images_path + file)
            data_y.append(masks_path + file)
    return(data_x, data_y)

In [124]:
(images, masks) = make_train_test_data(train_parts_images,train_parts_masks)

In [126]:
# Функция для аугментации данных: увеличим число данных за счет 
# рандомного поворота на предельные 12 градусоа
def augment_data(images, masks, save_imgs_path,save_masks_path, augment=True):
    #size = (1624, 1232)
    for idx, (x, y) in tqdm(enumerate(zip(images,masks)), total=len(images)):
        name = x.split('/')[-1].split('.')[0]
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = cv2.imread(y, cv2.IMREAD_COLOR)
        
        if augment == True:
            aug = Rotate(limit=12, p=1)
            augmented = aug(image=x, mask=y)
            x3 = augmented["image"]
            y3 = augmented["mask"]
            
            X = [x, x3]
            Y = [y, y3]
        else:
            X = [x]
            Y = [y]
        index = 0
        for i, m in zip(X,Y):
            tmp_image_name = name + '_' + str(index) + '.png'
            tmp_mask_name = name + '_' + str(index) + '.png'
            
            image_path = str(Path(save_imgs_path, tmp_image_name))
            mask_path = str(Path(save_masks_path, tmp_mask_name))
            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)
            
            index += 1

In [128]:
# После аугментации изображения сохранятся в директории "content_train/aug_images",
# маски сохранятся в директории "content_train/aug_masks"
augment_data(images,masks, aug_images, aug_masks, augment=True)